### Understand the code step by step

In [1]:
import numpy as np

In [2]:
input_texts = ['hi Dan', 'bye', 'never']
target_texts = ['salut Dan', 'au revoir', 'jamais']
input_characters = set()
target_characters = set()

In [3]:
max_encoder_seq_length = max([len(txt) for txt in input_texts])
print(max_encoder_seq_length)
min_encoder_seq_length = min([len(txt) for txt in input_texts])
print(min_encoder_seq_length)
max_decoder_seq_length = max([len(txt) for txt in target_texts])

6
3


In [4]:
for input_text in input_texts:
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)

for target_text in target_texts:
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)


input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
print(input_characters)
print(target_characters)
for i, char in enumerate(input_characters):
    print("i, char -> ", i, char)

[' ', 'D', 'a', 'b', 'e', 'h', 'i', 'n', 'r', 'v', 'y']
[' ', 'D', 'a', 'e', 'i', 'j', 'l', 'm', 'n', 'o', 'r', 's', 't', 'u', 'v']
i, char ->  0  
i, char ->  1 D
i, char ->  2 a
i, char ->  3 b
i, char ->  4 e
i, char ->  5 h
i, char ->  6 i
i, char ->  7 n
i, char ->  8 r
i, char ->  9 v
i, char ->  10 y


In [5]:
input_token_index = dict([char, i] for i, char in enumerate(input_characters))
target_token_index = dict([char, i] for i, char in enumerate(target_characters))

print("input_token_index -> ", input_token_index)
print("target_token_index -> ", target_token_index)


input_token_index ->  {' ': 0, 'D': 1, 'a': 2, 'b': 3, 'e': 4, 'h': 5, 'i': 6, 'n': 7, 'r': 8, 'v': 9, 'y': 10}
target_token_index ->  {' ': 0, 'D': 1, 'a': 2, 'e': 3, 'i': 4, 'j': 5, 'l': 6, 'm': 7, 'n': 8, 'o': 9, 'r': 10, 's': 11, 't': 12, 'u': 13, 'v': 14}


In [6]:
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)

encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32")
decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32")
decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32")

print("input texts length -> ", len(input_texts))
print("max encoder seq length -> ", max_encoder_seq_length)
print("num encoder tokens -> ", num_encoder_tokens)
print(encoder_input_data) # prints 4x16x17 array of zeros
#meaning -> 4 dimensions, 16 rows & 17 columns

input texts length ->  3
max encoder seq length ->  6
num encoder tokens ->  11
[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]]


In [7]:
print(tuple(zip(input_texts, target_texts)))

(('hi Dan', 'salut Dan'), ('bye', 'au revoir'), ('never', 'jamais'))


In [8]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t + 1:, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
            if t > 0:
                # decoder_target_data will be ahead by one timestep
                # and will not include the start character.
                decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1:, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0


print("encoder_input_data -> ", encoder_input_data)

encoder_input_data ->  [[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]]

 [[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
  [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
  [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
  [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]]
